# Set up environment

In [ ]:
import sys
print(sys.prefix)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import urllib, os,sys, pdfplumber, glob, requests, wordcloud, re, dateparser

# Set up working dir

In [ ]:
base_dir = os.path.realpath('../..')
print(base_dir)
data_dir = base_dir + '\Data'

In [ ]:
in_dir = data_dir + '\\TK_commissieVWS\\auto_download_20230118'

In [ ]:
results_dir = base_dir + '\\Results\\dataset_20230118'

## Load speaking turns data

In [ ]:
vws_data = pd.read_csv(in_dir + '\\speaking_turns_coded_labeled_noSEGV.csv', index_col = 0)

In [ ]:
vws_data['month'] = vws_data['date'].apply(lambda x: x[:7])

In [ ]:
vws_data.head(2)

In [ ]:
vws_data.tail(2)

In [ ]:
cabinets = vws_data['cabinet'].unique().tolist()
print(cabinets)

In [ ]:
cabinet_dates = pd.DataFrame({
    'Balkenende-IV':'2006-11-30',
    'Rutte-I':'2010-06-17',
    'Rutte-II':'2012-09-20',
    'Rutte-III':'2017-03-23',
    'Rutte-IV':'2021-03-31'
}
    , index = [0]).T.reset_index()
cabinet_dates.columns = ['name','date']
cabinet_dates

In [ ]:
events = ['preventieakkoord']

In [ ]:
vws_data.shape

## Inductively investigating peaks and troughs in LEEF/SDOH contributions

In [ ]:
nice_cat_labels = {'LEEF':'Lifestyle','SDOH':'SDOH'}

import matplotlib.ticker as mtick
import matplotlib.dates as mdates

Average by month:

In [ ]:
month_data = vws_data[['date','month','year','cabinet','LEEF','SDOH','days_since_installation','preventieakkoord']
                   ].groupby(by=['month'], as_index=False).mean().copy()
month_data['month_dt'] = pd.to_datetime(month_data['month'], infer_datetime_format=True)

xtickstep = 3
fig, axes = plt.subplots(1,1,figsize=[14,4])
sns.lineplot(data = month_data, x = 'month_dt', y = 'LEEF', label = 'Lifestyle', )
sns.lineplot(data = month_data, x = 'month_dt', y = 'SDOH', label = 'SDOH')
axes.set(ylabel = 'Parliamentary contributions referencing theme',
        title = 'When does Commissie VWS talk about lifestyle/SDOH?')
axes.yaxis.set_major_formatter(mtick.PercentFormatter(xmax = 1, decimals = 0))

# Add labels for events
for ei,event in enumerate([events[0]]):
    axes.plot(month_data.loc[month_data[event]==1,'month_dt'],
          -.02*np.ones(len(month_data.loc[month_data[event]==1,'month_dt'])), label = event)
axes.legend();

# Add dotted lines for cabinets
for ci,cab in cabinet_dates.iterrows():
    month = cab['date'][:7]
#     print(month)
    if (month >= month_data['month'].min()) and (month <= month_data['month'].max()):
        axes.plot([pd.to_datetime(month), pd.to_datetime(month)], [-0.02,.4], 'k--')
        axes.text(pd.to_datetime(month), 0.35, '  '+cab['name'], ha='left')

# Add peak indicators
for ci,cat in enumerate(['LEEF','SDOH']):
    cutoff = month_data[cat].mean() + month_data[cat].std()*3
    peak_data = month_data.loc[month_data[cat]>=cutoff,:].copy()
    sns.scatterplot(data = peak_data, x = 'month_dt', y = cat, s = 100,
                    label = '%s > 3*s.d.'%(nice_cat_labels[cat]), zorder = 10, edgecolor = 'k')
    peak_text = vws_data.loc[(vws_data['month'].isin(peak_data['month'].unique()) & 
                              (vws_data[cat]==1)),:]
    peak_data.to_csv(results_dir + '//Peak_data_%s.csv'%cat)
    peak_text.to_csv(results_dir + '//Peak_snippets_%s.csv'%cat)
    print('Peak text snippets saved for %s'%cat)

# Show every nth month label
axes.xaxis.set_major_locator(mdates.YearLocator())
axes.xaxis.set_major_formatter(mdates.DateFormatter('%m-%Y'))
axes.xaxis.set_minor_locator(mdates.MonthLocator())
plt.xlim([pd.to_datetime('06-2008', format = '%m-%Y'),pd.to_datetime('12-2022', format = '%m-%Y')])
axes.set(xlabel = 'Calendar date (by month)')

fig.savefig(results_dir + '//Time_course_LEEF_SDOH_by_month.pdf', bbox_inches = 'tight', transparent = True)

In [ ]:
peak_data

# Testing lifestyle drift

In [ ]:
# Settings:
n_perm = 10000
time_var = 'days_since_installation'
time_var_pretty = 'Days since installation'
center_estimator = np.median
cent_est_label = 'median'

In [ ]:
fig, axes = plt.subplots(nrows = len(cabinets[1:]), ncols = 1, figsize = [12,3*len(cabinets[1:])])

colors = {'LEEF':sns.color_palette('tab10')[0],
          'SDOH':sns.color_palette('tab10')[1],
         'Neither':sns.color_palette('Greys',2)[1]}
labels = {'LEEF':'Lifestyle',
          'SDOH':'SDOH',
          'Neither':'Control'}
linestyles = {'LEEF':'-',
             'SDOH':'-',
             'Neither':'--'}
plot_cats = ['LEEF','SDOH','Neither']

all_bars = pd.DataFrame(columns = ['cab_i','cabinet','cat','bar_height'])

for cab_i,cabinet in enumerate(cabinets[1:]):
    
    print(cabinet)
    
    cab_dat = vws_data.loc[vws_data.cabinet==cabinet,:].copy().reset_index(drop=True)
    time_obs = cab_dat[time_var].values
    
    leef_obs = cab_dat['LEEF'].values
    sdoh_obs = cab_dat['SDOH'].values
    neither_obs = 1 - (cab_dat['LEEF'] | cab_dat['SDOH']).values
    obs_dict = {'LEEF':leef_obs,
               'SDOH':sdoh_obs,
               'Neither':neither_obs}
    
    ax = axes[cab_i]
    
    # Histograms
    for cat_i, cat in enumerate(plot_cats):
        histinfo = ax.hist(time_obs[obs_dict[cat]==1], bins = np.arange(0,1600,90), alpha = 1,
                    density = True, label = labels[cat], color = colors[cat], histtype = 'step', lw = 1)
        all_bars_tmp = pd.DataFrame(histinfo[0], columns = ['bar_height'])
        all_bars_tmp['cab_i'] = cab_i
        all_bars_tmp['cabinet'] = cabinet
        all_bars_tmp['cat'] = cat
        all_bars_tmp['x'] = np.arange(0,1600,90)[:-1]
        all_bars_tmp['x'] = all_bars_tmp['x'].astype(int)
        all_bars = all_bars.append(all_bars_tmp)
        
    # Medians
    line_lim = np.multiply(ax.get_ylim(),1.2)
    for cat in plot_cats:
        center_time = center_estimator(time_obs[obs_dict[cat]==1])
        ax.plot([center_time, center_time],line_lim,
                ls = linestyles[cat], color = colors[cat], lw = 3, label = '%s(%s)'%(cent_est_label, labels[cat]))
        if cat in ['LEEF','SDOH']:
            # Add p-value:
            cat_obs_perm = np.copy(obs_dict[cat])
            perm_center_time = []
            for p in range(n_perm):
                np.random.shuffle(cat_obs_perm)
                perm_center_time.append(center_estimator(time_obs[cat_obs_perm==1]))
            if center_time >= np.mean(perm_center_time):
                p_val = np.mean(center_time < perm_center_time)
            else:
                p_val = np.mean(center_time > perm_center_time)
            p_val_2sided = 2.*p_val
            p_val_corrected_2sided = p_val_2sided*4. # Bonferroni correction across 4 cabinets
            bbox = {'fc': '1', 'edgecolor':colors[cat], 'pad': .5, 'boxstyle': 'round'}
            ypos = line_lim[1]*1.1
            if cat == 'SDOH':
                ypos = line_lim[1]*.9
            p_text = 'p = %.4f'%p_val_corrected_2sided if p_val_corrected_2sided < 0.05 else 'n.s.'
            ax.text(center_time, ypos, p_text,
                    color = colors[cat], fontdict = {'ha': 'center', 'va': 'center', 'bbox': bbox})
            ax.set_ylim([line_lim[0], line_lim[1]*1.3])
    
    # Look
    ax.set(title = cabinet, xlabel = time_var_pretty, yticks = [], ylabel = 'Density')
    ax.legend()

plt.tight_layout()
# fig.savefig(results_dir + '//Permutation_test_median_topical_delay.pdf',
#             bbox_inches = 'tight', transparent = True)